In [1]:
from matplotlib import pyplot as plt
from scipy.special import softmax
from itertools import product
import numpy as np
from utils import *
from model import *
import time
import seaborn as sns
from glob import glob as glob
sns.set_context('talk')

%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
gsname = 'gs1029'

### full gs data

In [ ]:
dfpathL = glob('data/%s/*'%gsname)
dfL = []
for p in dfpathL[:-1]:
  try:
    df_ = pd.read_csv(p)
    dfL.append(df_)
  except:
    print('error',p)
    continue
datadf = pd.concat(dfL).drop(columns='Unnamed: 0',)
# datadf ## full data

In [ ]:
datadf

In [ ]:
datadf.columns
paramL = ['concentration', 'stickiness_wi', 'stickiness_bt',
       'sparsity', 'pvar', 'lrate', 'lratep', 'decay_rate']

### compute df metrics, form summary df

In [ ]:
#### package this. set option to load presaved df
## compute summary df
groupvars = paramL 
gsdf_group = datadf.groupby(groupvars)
dfL = []
for params_i,df_i in gsdf_group:
  dataD = {**dict(zip(groupvars,params_i))}
  ## loop conditions (BIEML)
  for cond_i,df_c in df_i.groupby('cond'):
    ## compute metrics
    testacc = np.mean(df_c.acc[-40:])
    ## populate dataD
    dataD['testacc-%s'%cond_i[0]] = testacc
  # 
  dfL.append(pd.DataFrame(index=[0],data=dataD))
gsdf = pd.concat(dfL)


In [ ]:
### save gsdf: takes really long to run
gsdf.to_csv('data/%s-summdf.csv'%gsname)
### 

In [ ]:
## subset of dataframe containing relevant params
Btest = gsdf.loc[:,'testacc-b'] > 0.8
Itest1 = gsdf.loc[:,'testacc-i'] > 0.5
Itest2 = gsdf.loc[:,'testacc-i'] < 0.65
Etest = gsdf.loc[:,'testacc-e'] > 0.8
Mtest = gsdf.loc[:,'testacc-m'] > 0.8
Ltest = gsdf.loc[:,'testacc-l'] > 0.8
MItest = gsdf.loc[:,'testacc-m'] - gsdf.loc[:,'testacc-i'] > 0.1
LItest = gsdf.loc[:,'testacc-l'] - gsdf.loc[:,'testacc-i'] > 0.1

C = Btest&Itest2&Itest1&MItest
sgsdf = gsdf[C]
sgsdf

### plt single condition

In [ ]:
## select single param for plotting full acc trace
row = sgsdf.iloc[-1]

query_str = " and ".join(
  ["%s == %f"%(k,v) for k,v in dict(row[:8].round(4)).items()]
)

## df of single parameter condition
cond_df = datadf.round(4).query(query_str)

In [ ]:
f,axar = plt.subplots(1,1,figsize=(20,8))
ax=axar
for c,cdf in cond_df.groupby('cond'):
  ax.plot(np.arange(200),cdf.acc,label=c)
ax.set_ylim(0,1)
plt.legend()